In [1]:
import pandas as pd

In [2]:
def read_bid_ask_data(ask_fname : str, bid_fname : str):
    """Reads and combines the bid and ask csv files into a single OHLCV dataframe."""
    df_ask = pd.read_csv(ask_fname, infer_datetime_format=True)
    df_bid = pd.read_csv(bid_fname, infer_datetime_format=True)

    df_avg = (df_bid[["Open", "High", "Low", "Close", "Volume"]]+ df_ask[["Open", "High", "Low", "Close", "Volume"]]) / 2.0
    df_avg["time"] = df_ask["Local time"]
    df_avg = df_avg[df_avg["Volume"] > 0].reset_index()
    df_avg["time"] = df_avg["time"].str.replace(r'.\d{3} GMT-\d\d\d\d', '', regex = True) ## Strip ms and GMT TZ in time column
    if "index" in list(df_avg):
        df_avg.drop(labels = "index", axis = 1, inplace = True)
    df_avg["time"] = pd.to_datetime(df_avg["time"], format='%d.%m.%Y %H:%M:%S')
    df_avg.set_index(keys="time", inplace = True)
    # df_avg.columns= df_avg.columns.str.lower() 
    ## Results in ValueError: `data` must be a pandas.DataFrame with columns 'Open', 'High', 'Low', 'Close', and (optionally) 'Volume' for backtesting.py
    return df_avg

In [3]:
### DataFrame Slicing based on nr. of rows on 1m dataframe
def slice_df_by_1m_rows(df : pd.DataFrame, nr_days_to_slice : int):
    """Slice the historical dataframe from most recent to the nr. of days specified"""
    mins_per_day = 24 * 60
    nr_days_to_slice = 365 * mins_per_day
    df = df.iloc[-nr_days_to_slice:].reset_index(drop = True)
    return df

In [4]:
gu_ask_fname1 = "/Users/dilip.rajkumar/Documents/QubitQuants/GBPUSD_Candlestick_1_M_ASK_27.08.2019-27.08.2022.csv"
gu_bid_fname2 = "/Users/dilip.rajkumar/Documents/QubitQuants/GBPUSD_Candlestick_1_M_BID_27.08.2019-27.08.2022.csv"
df = read_bid_ask_data(gu_ask_fname1, gu_bid_fname2)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1122468 entries, 2019-08-27 00:00:00 to 2022-08-26 16:59:00
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   Open    1122468 non-null  float64
 1   High    1122468 non-null  float64
 2   Low     1122468 non-null  float64
 3   Close   1122468 non-null  float64
 4   Volume  1122468 non-null  float64
dtypes: float64(5)
memory usage: 51.4 MB


In [6]:
display(df)

,Open,High,Low,Close,Volume
time,,,,,
2019-08-27 00:00:00,1.221095,1.221100,1.221035,1.221080,3.627000e+07
2019-08-27 00:01:00,1.221090,1.221105,1.221075,1.221085,4.071500e+07
2019-08-27 00:02:00,1.221090,1.221105,1.221075,1.221095,4.669500e+07
2019-08-27 00:03:00,1.221095,1.221215,1.221080,1.221200,6.567500e+07
2019-08-27 00:04:00,1.221215,1.221310,1.221185,1.221300,6.968500e+07
...,...,...,...,...,...
2022-08-26 16:55:00,1.173970,1.174070,1.173775,1.174015,2.849700e+08
2022-08-26 16:56:00,1.174035,1.174145,1.173420,1.173920,5.481400e+08
2022-08-26 16:57:00,1.173925,1.173955,1.173820,1.173940,1.303600e+08


In [7]:
df.to_hdf("GU_OHLCV_3Y.h5", key='df', mode='w')